# 21.0 简介

本章指出，前20章介绍了从数据获取到模型创建的完整流程，但要让模型产生实际价值，需要将其整合到应用中。因此，训练后保存模型并在需要时加载是最后一章的核心内容。

# 21.1 保存和加载一个scikit_learn模型

In [1]:
from numpy import number
# 加载库
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
import joblib

from notebooks.Chapter16_Logistic_Regression.ch16_Logistic_regression import new_observation

from notebooks.Chapter20_Neural_Network.ch20_Neural_Network import number_of_features

# 加载数据
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 创建决策树分类器对象
classifier = RandomForestClassifier()

# 训练模型
model = classifier.fit(features, target)

# 把模型保存为pickle文件
joblib.dump(model, 'model.pkl')

['model.pkl']

In [2]:
# 从文件中加载模型
classifer =joblib.load('model.pkl')

In [3]:
# 创建新的样本
new_observation= [[5.2,3.2,1.1,0.1]]

# 预测样本的分类
classifer.predict(new_observation)

array([0])

- 讨论：
  - `joblib` 专门用于处理包含大 NumPy 数组的 `scikit-learn` 模型，比标准 `pickle` 更高效。
  - **版本兼容性问题**：不同版本的 `scikit-learn` 之间模型可能不兼容。建议在保存文件名中包含 `scikit-learn` 的版本号（如 `model_0.11.pkl`），以避免后续加载时出错。

# 21.2 保存和加载Keras模型

In [6]:
#加载库
import numpy as np
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers
from keras.models import load_model

#设置随机种子
np.random.seed(0)

# 设置想要的特征数量
number_of_features = 1000

# 从影评数据中加载数据和目标向量
(train_data,train_target),(test_data,test_target) = imdb.load_data(num_words=number_of_features)

# 把影评数据转换为独热编码
tokenizer = Tokenizer(num_words=number_of_features)
train_features = tokenizer.sequences_to_matrix(train_data, mode='binary')
test_features = tokenizer.sequences_to_matrix(test_data, mode='binary')

# 启动神经网络
network = models.Sequential()

# 添加使用ReLU激活函数的全连接层
network.add(layers.Dense(units = 16,
                         activation = 'relu',
                         input_shape=(number_of_features,)))

# 添加使用sigmoid激活函数的全连接层
network.add(layers.Dense(units = 1,activation='sigmoid'))

# 编译神经网络
network.compile(loss='binary_crossentropy',#交叉熵
                optimizer='adam',#优化器
                metrics=['accuracy'])# 将准确率作为性能指标

# 训练神经网络
history = network.fit(train_features,
                      train_target,
                      epochs=3,
                      verbose=0,
                      batch_size=100,
                      validation_data=(test_features, test_target))
# 保存神经网络
network.save('model.keras')

In [7]:
# 加载神经网络
network = load_model('model.keras')

- 讨论：
  - Keras **不推荐**使用 pickle 格式，而是推荐 HDF5 格式。
  - HDF5 文件不仅包含模型的结构和训练好的权重（用于预测），还包含损失函数、优化器设置及当前状态（用于重新训练）。